# ⚙️ ETAPA 3: PROCESSAMENTO COM DOCLING

## 🎯 **O que esta etapa faz**
Extrai e estrutura o conteúdo de todos os documentos baixados, transformando PDFs, DOCs e outros formatos em texto estruturado pronto para análise.

## 🤔 **Por que esta etapa é necessária**
Documentos vêm em formatos diferentes (PDF, DOCX, MD). Precisamos:
- 📄 **Extrair texto puro** de cada documento
- 🏗️ **Preservar estrutura** (títulos, seções, tabelas)
- 🔍 **Usar OCR** quando necessário (imagens, PDFs escaneados)
- 📊 **Coletar metadados** de estrutura e conteúdo

## ⚙️ **Como funciona**
1. **Analisa cada documento** para determinar o melhor método de extração
2. **Aplica Docling** para extrair conteúdo estruturado
3. **Usa OCR** em PDFs escaneados ou com baixa qualidade de texto
4. **Preserva hierarquia** de títulos e seções
5. **Gera texto limpo** removendo formatação desnecessária
6. **Salva conteúdo estruturado** e texto puro separadamente

## 📊 **Parâmetros que você pode ajustar**
- `use_ocr`: Habilitar OCR automático (padrão: True)
- `ocr_confidence_threshold`: Limite de confiança OCR (padrão: 0.7)
- `preserve_structure`: Manter hierarquia de títulos (padrão: True)
- `clean_text`: Remover formatação extra (padrão: True)
- `extract_tables`: Processar tabelas separadamente (padrão: True)

## 👁️ **O que esperar de saída**
- 📁 **Pasta `processed/`** com conteúdo estruturado
- 📄 **Arquivos `*_content.json`** com estrutura completa
- 📝 **Arquivos `*_text.txt`** com texto puro extraído
- 📊 **Metadados** de extração (confiança, método usado, etc.)
- 📈 **Estatísticas** de processamento por tipo de arquivo

## ⚠️ **Pontos importantes**
- **Requer Etapa 2** (Coleta GitLab) concluída primeiro
- **Docling** deve estar instalado e funcional
- **OCR** pode ser lento para documentos grandes
- **Modo independente** disponível para testes

## 🛡️ Verificação de Dependências

In [ ]:
# 🛡️ VERIFICAÇÃO AUTOMÁTICA DE DEPENDÊNCIAS
import sys
import os
from pathlib import Path
from datetime import datetime

# Adicionar biblioteca ao path
sys.path.insert(0, str(Path().parent / "src"))

from pipeline_utils import (
    PipelineState, 
    check_prerequisites, 
    show_pipeline_progress
)

print("⚙️ ETAPA 3: PROCESSAMENTO COM DOCLING")
print("=" * 60)
print(f"🕒 Iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verificar dependências
CURRENT_STAGE = 3
prerequisites_ok = check_prerequisites(CURRENT_STAGE)

if not prerequisites_ok:
    print("\n❌ ERRO: Dependências não atendidas!")
    print("📋 Execute primeiro as etapas anteriores:")
    print("   1. 01_FUNDACAO_PREPARACAO.ipynb")
    print("   2. 02_COLETA_GITLAB.ipynb")
    
    show_pipeline_progress()
    raise RuntimeError("Dependências não atendidas - execute etapas anteriores primeiro")

print("\n✅ Dependências verificadas - pode prosseguir")
print("📈 Progresso atual do pipeline:")
show_pipeline_progress()

## 📋 Carregamento de Dados

In [ ]:
# 📋 CARREGAMENTO DOS DADOS DA ETAPA ANTERIOR
print("📋 Carregando dados da etapa anterior...")

# Carregar dados da etapa 2 (GitLab)
state = PipelineState()
try:
    documents_data = state.load_stage_data(2)
    print("✅ Dados carregados da Etapa 2 (GitLab)")
    print(f"   📄 Documentos disponíveis: {len(documents_data['local_files'])}")
    print(f"   📁 Diretório: {documents_data['next_stage_instructions']['documents_directory']}")
except Exception as e:
    print(f"❌ Erro ao carregar dados: {e}")
    raise

print("\n" + "=" * 60)

## 📋 Configuração de Processamento

In [ ]:
# 📋 CONFIGURAÇÃO DOS PARÂMETROS DE PROCESSAMENTO
print("📋 Configurando parâmetros de processamento Docling...")

# 🎯 PARÂMETROS AJUSTÁVEIS
processing_config = {
    # Docling
    "use_ocr": True,
    "ocr_confidence_threshold": 0.7,
    "preserve_structure": True,
    "clean_text": True,
    "extract_tables": True,
    "extract_images": False,  # Por enquanto, focar em texto
    
    # Processamento
    "batch_size": 4,  # Processar até 4 documentos simultaneamente
    "timeout_per_doc_seconds": 300,  # 5 minutos por documento
    "skip_large_files_mb": 100,  # Pular arquivos muito grandes
    
    # Saída
    "output_dir": "../pipeline_data/processed",
    "save_intermediate": True,  # Salvar resultados intermediários
    "overwrite_existing": True
}

print("\n📊 Parâmetros de processamento configurados:")
print(f"   🔍 OCR habilitado: {processing_config['use_ocr']}")
print(f"   📊 Limite confiança OCR: {processing_config['ocr_confidence_threshold']}")
print(f"   🏗️ Preservar estrutura: {processing_config['preserve_structure']}")
print(f"   🧹 Limpar texto: {processing_config['clean_text']}")
print(f"   📋 Extrair tabelas: {processing_config['extract_tables']}")
print(f"   📦 Lote: {processing_config['batch_size']} documentos simultâneos")
print(f"   ⏱️ Timeout: {processing_config['timeout_per_doc_seconds']} segundos por documento")
print(f"   📁 Saída: {processing_config['output_dir']}")

# Preparar diretório de saída
from pipeline_utils import ensure_directory
ensure_directory(processing_config["output_dir"])

print("\n✅ Configuração de processamento pronta")

## 📄 Análise dos Documentos de Entrada

In [ ]:
# 📄 ANÁLISE DOS DOCUMENTOS PARA PROCESSAMENTO
print("📄 Analisando documentos para processamento...")

from collections import defaultdict
from pipeline_utils import format_file_size

# Analisar arquivos de entrada
files_to_process = documents_data["local_files"]
print(f"\n📊 Total de documentos: {len(files_to_process)}")

# Estatísticas por tipo
type_stats = defaultdict(lambda: {"count": 0, "total_size": 0, "files": []})
total_size = 0
processable_files = []
skipped_files = []

for file_info in files_to_process:
    file_path = Path(file_info["local_path"])
    file_ext = file_info["extension"]
    file_size = file_info["size_bytes"]
    file_size_mb = file_size / (1024 * 1024)
    
    # Verificar se arquivo existe
    if not file_path.exists():
        print(f"   ⚠️ Arquivo não encontrado: {file_path.name}")
        skipped_files.append((file_path.name, "file_not_found"))
        continue
    
    # Verificar tamanho
    if file_size_mb > processing_config["skip_large_files_mb"]:
        print(f"   ⏭️ Pulado (muito grande): {file_path.name} ({file_size_mb:.1f} MB)")
        skipped_files.append((file_path.name, "too_large"))
        continue
    
    # Arquivo OK para processamento
    type_stats[file_ext]["count"] += 1
    type_stats[file_ext]["total_size"] += file_size
    type_stats[file_ext]["files"].append(file_info)
    total_size += file_size
    processable_files.append(file_info)
    
    print(f"   ✅ {file_path.name} ({format_file_size(file_size)}) - {file_ext}")

print(f"\n📈 Resumo da análise:")
print(f"   ✅ Processáveis: {len(processable_files)}")
print(f"   ⏭️ Pulados: {len(skipped_files)}")
print(f"   📏 Tamanho total: {format_file_size(total_size)}")

print(f"\n📊 Por tipo de arquivo:")
for ext, stats in sorted(type_stats.items()):
    count = stats["count"]
    size = format_file_size(stats["total_size"])
    avg_size = format_file_size(stats["total_size"] / count) if count > 0 else "0 B"
    print(f"   {ext}: {count} arquivos, {size} total, {avg_size} médio")

if skipped_files:
    print(f"\n⚠️ Arquivos pulados ({len(skipped_files)}):")
    for filename, reason in skipped_files:
        reason_text = {
            "file_not_found": "não encontrado",
            "too_large": "muito grande"
        }.get(reason, reason)
        print(f"   - {filename}: {reason_text}")

if not processable_files:
    print("\n❌ Nenhum arquivo disponível para processamento")
    raise ValueError("Nenhum documento para processar")

print("\n✅ Análise concluída - pronto para processamento")

## ⚙️ Processamento com Docling

In [ ]:
# ⚙️ EXECUÇÃO DO PROCESSAMENTO COM DOCLING
print("⚙️ Iniciando processamento com Docling...")

from docling_utils import extract_content_from_file, batch_process_documents
import json
from datetime import datetime

# Preparar lista de arquivos para processamento
file_paths = [file_info["local_path"] for file_info in processable_files]

print(f"\n🚀 Processando {len(file_paths)} documentos...")
print("=" * 60)

# Executar processamento em lote
start_time = datetime.now()

processing_results = batch_process_documents(
    file_list=file_paths,
    output_dir=processing_config["output_dir"]
)

end_time = datetime.now()
processing_duration = (end_time - start_time).total_seconds()

print("\n" + "=" * 60)
print("📊 RESULTADO DO PROCESSAMENTO:")

# Contar sucessos e falhas
successful_results = [r for r in processing_results if "error" not in r]
failed_results = [r for r in processing_results if "error" in r]

print(f"   📄 Total processado: {len(processing_results)}")
print(f"   ✅ Sucessos: {len(successful_results)}")
print(f"   ❌ Falhas: {len(failed_results)}")
print(f"   ⏱️ Tempo total: {processing_duration:.1f} segundos")

if successful_results:
    avg_time = processing_duration / len(successful_results)
    print(f"   📈 Tempo médio: {avg_time:.1f} segundos por documento")

# Mostrar resultados de sucesso
if successful_results:
    print("\n📁 Documentos processados com sucesso:")
    for result in successful_results[:10]:  # Mostrar apenas os primeiros 10
        filename = Path(result["source_file"]).name
        method = result["processing_info"]["method"]
        confidence = result["processing_info"]["confidence_score"]
        print(f"   ✅ {filename} (método: {method}, confiança: {confidence:.2f})")
    
    if len(successful_results) > 10:
        remaining = len(successful_results) - 10
        print(f"   ... e mais {remaining} documentos")

# Mostrar falhas se houver
if failed_results:
    print(f"\n⚠️ Documentos com problemas ({len(failed_results)}):")
    for result in failed_results[:5]:  # Mostrar apenas os primeiros 5
        filename = Path(result["source_file"]).name
        error = result["error"]
        print(f"   ❌ {filename}: {error}")
    
    if len(failed_results) > 5:
        remaining = len(failed_results) - 5
        print(f"   ... e mais {remaining} erros")

print("\n✅ Processamento concluído!")

## 📊 Validação dos Resultados

In [ ]:
# 📊 VALIDAÇÃO DOS RESULTADOS PROCESSADOS
print("📊 Validando resultados do processamento...")

from docling_utils import validate_processed_content
import os

# Validar conteúdo processado
validation_result = validate_processed_content(processing_config["output_dir"])

if validation_result["valid"]:
    print("\n✅ Validação bem-sucedida")
    print(f"   📄 Arquivos JSON: {validation_result['json_files_count']}")
    print(f"   📝 Arquivos TXT: {validation_result['text_files_count']}")
    print(f"   📊 Total de arquivos: {validation_result['total_files']}")
else:
    print(f"\n❌ Problema na validação: {validation_result.get('error', 'Erro desconhecido')}")

# Coletar estatísticas detalhadas
processed_dir = Path(processing_config["output_dir"])
content_files = list(processed_dir.glob("*_content.json"))
text_files = list(processed_dir.glob("*_text.txt"))

print(f"\n📁 Arquivos gerados no diretório {processed_dir}:")
print(f"   📄 Conteúdo estruturado: {len(content_files)} arquivos JSON")
print(f"   📝 Texto puro: {len(text_files)} arquivos TXT")

# Verificar qualidade do texto extraído
text_quality_stats = {
    "total_characters": 0,
    "total_words": 0,
    "files_with_content": 0,
    "empty_files": 0
}

for text_file in text_files:
    try:
        with open(text_file, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        
        if content:
            text_quality_stats["total_characters"] += len(content)
            text_quality_stats["total_words"] += len(content.split())
            text_quality_stats["files_with_content"] += 1
        else:
            text_quality_stats["empty_files"] += 1
            print(f"   ⚠️ Arquivo vazio: {text_file.name}")
    except Exception as e:
        print(f"   ❌ Erro ao ler {text_file.name}: {e}")

if text_quality_stats["files_with_content"] > 0:
    avg_chars = text_quality_stats["total_characters"] / text_quality_stats["files_with_content"]
    avg_words = text_quality_stats["total_words"] / text_quality_stats["files_with_content"]
    
    print(f"\n📈 Qualidade do texto extraído:")
    print(f"   📝 Arquivos com conteúdo: {text_quality_stats['files_with_content']}")
    print(f"   🔤 Total de caracteres: {text_quality_stats['total_characters']:,}")
    print(f"   📖 Total de palavras: {text_quality_stats['total_words']:,}")
    print(f"   📊 Média por arquivo: {avg_chars:.0f} caracteres, {avg_words:.0f} palavras")
    
    if text_quality_stats["empty_files"] > 0:
        print(f"   ⚠️ Arquivos vazios: {text_quality_stats['empty_files']}")
else:
    print("\n❌ Nenhum conteúdo de texto foi extraído com sucesso")

# Preparar dados para próxima etapa
processed_files_info = []
for result in successful_results:
    content_file = result.get("content_file")
    text_file = result.get("text_file")
    
    if content_file and text_file:
        processed_files_info.append({
            "source_document": result["source_file"],
            "content_file": content_file,
            "text_file": text_file,
            "processing_method": result["processing_info"]["method"],
            "confidence_score": result["processing_info"]["confidence_score"],
            "structure_info": result["structure"]
        })

print(f"\n📋 Documentos preparados para próxima etapa: {len(processed_files_info)}")
print("✅ Validação concluída!")

## 💾 Salvamento dos Resultados

In [ ]:
# 💾 SALVAMENTO DOS RESULTADOS PARA PRÓXIMA ETAPA
print("💾 Salvando resultados do processamento...")

from pipeline_utils import get_timestamp

# Preparar dados de saída para próxima etapa
stage_output = {
    "stage_info": {
        "stage_number": 3,
        "stage_name": "docling",
        "completed_at": get_timestamp(),
        "status": "success" if len(successful_results) > 0 else "completed_with_warnings",
        "processing_duration_seconds": processing_duration
    },
    
    "processing_config": processing_config,
    
    "input_files": {
        "total_input_files": len(files_to_process),
        "processable_files": len(processable_files),
        "skipped_files": len(skipped_files)
    },
    
    "processed_documents": processed_files_info,
    
    "processing_results": {
        "successful_extractions": len(successful_results),
        "failed_extractions": len(failed_results),
        "total_processing_time_seconds": processing_duration
    },
    
    "text_quality": text_quality_stats,
    
    "statistics": {
        "files_processed": len(processing_results),
        "successful_extractions": len(successful_results),
        "failed_extractions": len(failed_results),
        "total_text_characters": text_quality_stats["total_characters"],
        "total_text_words": text_quality_stats["total_words"],
        "avg_processing_time_seconds": processing_duration / len(successful_results) if successful_results else 0,
        "content_files_generated": len(content_files),
        "text_files_generated": len(text_files)
    },
    
    "errors": [
        {
            "source_file": result["source_file"],
            "error": result["error"]
        }
        for result in failed_results
    ],
    
    "next_stage_instructions": {
        "processed_directory": processing_config["output_dir"],
        "text_files_to_chunk": [info["text_file"] for info in processed_files_info],
        "recommended_chunk_size": 500,
        "recommended_overlap": 100
    }
}

# Salvar usando PipelineState
state = PipelineState()
state.save_stage_data(3, stage_output)

# Marcar etapa como concluída
state.mark_stage_completed(3)

print(f"✅ Resultados salvos: {state.metadata_dir / 'stage_03_docling.json'}")
print(f"✅ Checkpoint criado: {state.checkpoints_dir / 'stage_03_completed.lock'}")

# Salvar relatório detalhado de processamento
processing_report_path = Path(processing_config["output_dir"]) / "processing_report.json"
with open(processing_report_path, 'w', encoding='utf-8') as f:
    json.dump({
        "timestamp": get_timestamp(),
        "config": processing_config,
        "results": processing_results,
        "quality_stats": text_quality_stats,
        "file_type_stats": dict(type_stats)
    }, f, indent=2, ensure_ascii=False, default=str)

print(f"📝 Relatório detalhado salvo: {processing_report_path}")

# Exibir resumo final
print("\n" + "=" * 60)
print("🎉 ETAPA 3 CONCLUÍDA COM SUCESSO!")
print("=" * 60)

print(f"📊 Resumo do Processamento Docling:")
print(f"   📄 Documentos processados: {len(successful_results)}/{len(processable_files)}")
print(f"   📝 Texto extraído: {text_quality_stats['total_words']:,} palavras")
print(f"   ⏱️ Tempo total: {processing_duration:.1f} segundos")
print(f"   📁 Arquivos gerados: {len(content_files)} JSON + {len(text_files)} TXT")
print(f"   📍 Localização: {processing_config['output_dir']}")

if failed_results:
    print(f"   ⚠️ Avisos: {len(failed_results)} documentos falharam")
else:
    print(f"   ✅ Status: Todos os processamentos bem-sucedidos")

print(f"\n🚀 Pronto para próxima etapa: 04_SEGMENTACAO_CHUNKS.ipynb")
print(f"📋 {len(processed_files_info)} documentos prontos para segmentação")

# Exibir progresso do pipeline
print("\n📈 Progresso do Pipeline:")
show_pipeline_progress()

---

## ✅ **Etapa 3 Concluída!**

### 🎯 **O que foi feito:**
- ✅ Documentos analisados por tipo e tamanho
- ✅ Conteúdo extraído com Docling (OCR quando necessário)
- ✅ Estrutura preservada (títulos, seções, tabelas)
- ✅ Texto limpo e metadados coletados
- ✅ Validação de qualidade realizada

### 🚀 **Próxima etapa:**
**`04_SEGMENTACAO_CHUNKS.ipynb`** - Segmentação inteligente de texto

### 📊 **Arquivos gerados:**
- `pipeline_data/processed/` - Conteúdo estruturado e texto extraído
- `pipeline_data/processed/*_content.json` - Estrutura completa de cada documento
- `pipeline_data/processed/*_text.txt` - Texto puro extraído
- `pipeline_data/metadata/stage_03_docling.json` - Metadados e estatísticas
- `pipeline_data/processed/processing_report.json` - Relatório detalhado
- `pipeline_data/checkpoints/stage_03_completed.lock` - Checkpoint de conclusão

### 📋 **Dados para próxima etapa:**
- **Texto estruturado:** Pronto para segmentação em chunks
- **Metadados de estrutura:** Seções, tabelas, confiança de extração
- **Qualidade validada:** Texto limpo e consistente

### 🔧 **Para executar próxima etapa:**
1. Abra o notebook `04_SEGMENTACAO_CHUNKS.ipynb`
2. Execute as células em sequência
3. Ou use o `00_PIPELINE_MASTER.ipynb` para execução automática

---

**⚙️ Conteúdo extraído e estruturado! Prontos para segmentação inteligente.**